In [85]:
import sqlite3
import pandas as pd
import csv
from sqlite3 import Error

# hide warnings
import warnings
warnings.filterwarnings('ignore')

### Creating Tables for the database

In [86]:
# Bookings table:
df = pd.read_csv('Bookings.csv')
df.columns = ['bookid', 'facid', 'memid', 'starttime', 'slots']
df.to_csv('Bookings.csv', index=False) 

In [87]:
# Facilities table:
df_F = pd.read_csv('Facilities.csv')
df_F.columns = ['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']
df_F.to_csv('Facilities.csv', index=False) 

In [88]:
# Members table:
df_M = pd.read_csv('Members.csv')
df_M.columns = ['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone', 'recommendedby', 'joindate']
df_M.to_csv('Members.csv', index=False) 

### Dump the csv into a local database called 'club.db'

In [89]:
!csv-to-sqlite -f Bookings.csv -f Facilities.csv -f Members.csv -o club.db

  [############------------------------]   33%Error on table Bookings: 
 table [Bookings] already exists
  [########################------------]   66%Error on table Facilities: 
 table [Facilities] already exists
  [####################################]  100%Error on table Members: 
 table [Members] already exists

Written 0 rows into 3 tables in 0.019 seconds


In [90]:
!ls

1520094343_sql_project.sql Members.csv
Bookings.csv               SQL_project.ipynb
Facilities.csv             club.db


### Connect to the database

In [91]:
con = sqlite3.connect('club.db')
c = con.cursor()

### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [92]:
rs = c.execute("SELECT * FROM Facilities WHERE membercost > 0")
df3 = pd.DataFrame(rs.fetchall())
df3.iloc[:,1]

0    Tennis Court 2
1    Massage Room 1
2    Massage Room 2
3      Squash Court
Name: 1, dtype: object

### Q2: How many facilities do not charge a fee to members?

In [93]:
rs = c.execute("SELECT count(*) FROM Facilities WHERE membercost = 0")
df3 = pd.DataFrame(rs.fetchall())
print(str(df3) + ' facilities')

   0
0  4 facilities


### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [113]:
rs = c.execute("SELECT facid, name, membercost, monthlymaintenance,  \
               CASE WHEN membercost > 0 AND membercost < 0.2*monthlymaintenance THEN 'yes'\
               ELSE NULL END AS facilities_charged \
               FROM Facilities")
df3 = pd.DataFrame(rs.fetchall())

In [116]:
df4 = df3[df3.iloc[:,4] == 'yes']
df4.columns = ['facid', 'facility_name', 'member_cost', 'monthly_maintainance', 'is_charged']
df4.iloc[:, 0:4]

,facid,facility_name,member_cost,monthly_maintainance
0,1,Tennis Court 2,5.0,200
3,4,Massage Room 1,9.9,3000
4,5,Massage Room 2,9.9,3000
5,6,Squash Court,3.5,80


### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [119]:
rs = c.execute("SELECT * \
                FROM Facilities \
                WHERE facid IN (1, 5)")
df3 = pd.DataFrame(rs.fetchall())

In [120]:
df3.columns = ['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']
df3

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question.

In [98]:
rs = c.execute("SELECT name, monthlymaintenance,  \
               CASE WHEN monthlymaintenance > 1000 THEN 'expensive'\
               ELSE 'cheap' END AS expensive_facilities \
               FROM Facilities")
df3 = pd.DataFrame(rs.fetchall())

In [99]:
df3

,0,1,2
0,Tennis Court 2,200,cheap
1,Badminton Court,50,cheap
2,Table Tennis,10,cheap
3,Massage Room 1,3000,expensive
4,Massage Room 2,3000,expensive
5,Squash Court,80,cheap
6,Snooker Table,15,cheap
7,Pool Table,15,cheap


### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution. 

In [100]:
rs = c.execute("SELECT firstname, surname \
                FROM Members \
                ORDER BY joindate")
df3 = pd.DataFrame(rs.fetchall())

In [101]:
df3.iloc[-1,:]

0    Darren
1     Smith
Name: 29, dtype: object

### Q7: How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [102]:
# Filter to get the 'facid" of Tennis Court from the Facilities table:
rs = c.execute("SELECT facid \
                FROM Facilities \
                WHERE name LIKE 'Tennis Court%'")
df_facid = pd.DataFrame(rs.fetchall())

In [103]:
df_facid

,0
0,1


In [104]:
# Select DISTINCT members who used facid = 1:
rs = c.execute("SELECT DISTINCT members.surname AS members_surname, \
                    members.firstname AS members_firstname \
                FROM Bookings bookings \
                INNER JOIN Members members \
                ON members.memid = bookings.memid \
                WHERE bookings.facid = 1 \
                ORDER BY members.surname")
df3 = pd.DataFrame(rs.fetchall())

In [105]:
'Tennis Court 2: ' + df3.iloc[:,0] + ' ' + df3.iloc[:,1]

0        Tennis Court 2: Bader Florence
1            Tennis Court 2: Baker Anne
2         Tennis Court 2: Baker Timothy
3            Tennis Court 2: Boothe Tim
4        Tennis Court 2: Butters Gerald
5            Tennis Court 2: Dare Nancy
6        Tennis Court 2: Farrell Jemima
7         Tennis Court 2: Farrell David
8             Tennis Court 2: Hunt John
9           Tennis Court 2: Jones David
10      Tennis Court 2: Joplette Janice
11         Tennis Court 2: Owen Charles
12    Tennis Court 2: Purview Millicent
13           Tennis Court 2: Rownam Tim
14     Tennis Court 2: Rumney Henrietta
15     Tennis Court 2: Sarwin Ramnaresh
16         Tennis Court 2: Smith Darren
17          Tennis Court 2: Smith Tracy
18           Tennis Court 2: Smith Jack
19      Tennis Court 2: Stibbons Ponder
20         Tennis Court 2: Tracy Burton
dtype: object

### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 
Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [106]:
rs = c.execute("SELECT facilities.name, bookings.memid, facilities.membercost, facilities.guestcost, bookings.slots, \
               CASE WHEN bookings.memid = 0 THEN facilities.guestcost*bookings.slots \
               ELSE facilities.membercost END AS booking_cost \
               FROM Bookings bookings \
               JOIN Facilities facilities \
               ON Bookings.facid = Facilities.facid \
               WHERE bookings.starttime LIKE '2012-09-14%' ")
               
df3 = pd.DataFrame(rs.fetchall())

In [107]:
df3.columns = ['fac_name', 'memid', 'membercost', 'guestcost', 'slots', 'this_cost']
df3.head(2)

,fac_name,memid,membercost,guestcost,slots,this_cost
0,Tennis Court 2,11,5.0,25.0,6,5.0
1,Tennis Court 2,8,5.0,25.0,6,5.0


In [108]:
df4 = df3[df3['this_cost'] > 30]

# According to 'memid', assign member name
guest_name = []
for index, row in df4.iterrows():
    if row['memid'] == 0:
        guest_name.append('guest')
        
df4['guest_name'] = guest_name

# sort according to 'this_cost' column:
df5 = df4.sort_values(['this_cost'], ascending=False)

df5['fac_name'] + ': ' + df5['guest_name'] + ' ' + df5['this_cost'].map(str)

23    Massage Room 2: guest 320.0
14    Massage Room 1: guest 160.0
17    Massage Room 1: guest 160.0
19    Massage Room 1: guest 160.0
3     Tennis Court 2: guest 150.0
2      Tennis Court 2: guest 75.0
25       Squash Court: guest 70.0
26       Squash Court: guest 35.0
28       Squash Court: guest 35.0
dtype: object

### Q9: This time, produce the same result as in Q8, but using a subquery.

In [109]:
rs = c.execute("SELECT sub.name, sub.booking_cost,\
                CASE WHEN sub.memid = 0 THEN 'guest' \
                ELSE 'member' END AS 'booking_name' \
                From (SELECT facilities.name, bookings.memid,\
                       CASE WHEN bookings.memid = 0 THEN facilities.guestcost*bookings.slots \
                       ELSE facilities.membercost END AS booking_cost \
                       FROM Bookings bookings \
                       JOIN Facilities facilities \
                       ON Bookings.facid = Facilities.facid \
                       WHERE bookings.starttime LIKE '2012-09-14%') sub \
               WHERE sub.booking_cost > 30 \
               ORDER BY booking_cost DESC")
               
df3 = pd.DataFrame(rs.fetchall())

df3.iloc[:,0] + ': $' + df3.iloc[:,1].map(str) + ' ' + df3.iloc[:,2]

0    Massage Room 2: $320.0 guest
1    Massage Room 1: $160.0 guest
2    Massage Room 1: $160.0 guest
3    Massage Room 1: $160.0 guest
4    Tennis Court 2: $150.0 guest
5     Tennis Court 2: $75.0 guest
6       Squash Court: $70.0 guest
7       Squash Court: $35.0 guest
8       Squash Court: $35.0 guest
dtype: object

### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [122]:
rs = c.execute("SELECT * \
                FROM(   SELECT sub.name, SUM(booking_cost) AS sum \
                   FROM (SELECT facilities.name, bookings.memid, facilities.membercost, facilities.guestcost, \
                           bookings.slots, \
                           CASE WHEN bookings.memid = 0 THEN facilities.guestcost*bookings.slots \
                           ELSE facilities.membercost END AS booking_cost \
                           FROM Bookings bookings \
                           JOIN Facilities facilities \
                           ON Bookings.facid = Facilities.facid) sub \
                   GROUP BY sub.name )\
               WHERE sum < 1000 \
               ORDER BY sum")
               
df3 = pd.DataFrame(rs.fetchall())

In [123]:
df3.columns = ['name', 'total_revenue']
df3

,name,total_revenue
0,Table Tennis,180.0
1,Snooker Table,240.0
2,Pool Table,270.0
